In [3]:
import pandas as pd
import numpy as np

In [9]:
def read_file(filename):
    base_path = r"C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Fact\\"
    return pd.read_csv(f"{base_path}\\{filename}")

In [10]:
# Read files
results = read_file("results.csv")
driverstandings = read_file("driver_standings.csv")
constructorstandings = read_file("constructor_standings.csv")
constructors = read_file("constructors.csv")
driver = read_file("drivers.csv")
races = read_file("races.csv")
laps = read_file("lap_times.csv")

In [11]:
# Create 'quarter' column
races["quarter"] = races.groupby(['year'])['round'].transform(lambda x: pd.qcut(x, 4, labels=range(1,5)))

# Group 'laps' dataframe
laps_gr = laps.groupby(['raceId', 'driverId'], as_index = False).milliseconds.mean()
laps_gr.columns = ['raceId', 'driverId', 'laptime_avg']

In [12]:
# Merge dataframes
data = (results
        .merge(driverstandings, on=["raceId", "driverId"], how="left")
        .merge(constructorstandings, on=["raceId", "constructorId"], how="left")
        .merge(races, on="raceId", how="left")
        .merge(driver, on="driverId", how="left")
        .merge(constructors, on="constructorId", how="left")
        .merge(laps_gr, on=['raceId', 'driverId'], how='left'))

In [13]:
# Column renaming
column_map = {
    'number_x': 'carnumber',
    'position_x': 'position',
    'positionOrder': 'end_race_position',
    'points_x': 'points_race',
    'time_x': 'race_duration',
    'points_y': 'points_driverstanding',
    'position_y': 'position_driverstanding',
    'wins_x': 'wins_driverstanding',
    'points': 'points_constructorstanding',
    'position': 'position_constructorstanding',
    'wins_y': 'wins_constructorstanding',
    'number_y': 'driver_number',
    'name_x': 'name_circuit',
    'nationality_x': 'nationality_driver',
    'name_y': 'name_constructor',
    'nationality_y': 'nationality_constructor',
    'rank': 'rank_fastestlap'
}
data.rename(columns=column_map, inplace=True)

In [14]:
columns_to_keep = ["resultId", "raceId", "driverId", 
                   "constructorId", "circuitId", "grid", "position",
                   "end_race_position", "points_race", "laps", "milliseconds", "fastestLap",
                   "rank_fastestlap", "fastestLapTime", "fastestLapSpeed", "points_driverstanding", "position_driverstanding",
                   "wins_driverstanding", "points_constructorstanding", "position_constructorstanding", "wins_constructorstanding",
                   "year", "round", "name_circuit", "date", "nationality_driver", "nationality_constructor", "driverRef",
                   "dob", "quarter", 'laptime_avg']
data = data[columns_to_keep]

In [15]:
# Replace values
data.replace([r"\N", r"\\N"], np.NaN, inplace=True)

# Change data types
data = data.astype({
    'fastestLap': "float",
    'milliseconds': "float",
    'rank_fastestlap': "float",
    'fastestLapSpeed': "float",
    "position": "float",
    'dob': "datetime64[ns]",
    'date': "datetime64[ns]"
})

In [16]:
# This section leverages vectorized operations
data['diff_months'] = ((data.dob - data.date)/np.timedelta64(1, 'M'))
data['race_months'] = pd.DatetimeIndex(data['date']).month
data['dob_quarter'] = pd.DatetimeIndex(data['dob']).quarter

In [17]:
# Create a deep copy to avoid modifying the original dataframe
data_copy = data[data['raceId'] != 1114].copy()

# Compute grid_end_diff
data["grid_end_diff"] = abs(data_copy["end_race_position"] - data_copy["grid"]).astype(int)

# Initialize an empty dataframe to store the results
df_overtakes = pd.DataFrame()

for year in sorted(data_copy['year'].unique()):
    # Filter data_copy for previous years
    previous_years_data = data[data['year'] < year]
    
    # Calculate mean overtakes per track for previous years, store in a dataframe
    previous_years_data['overtakes_per_track'] = previous_years_data.groupby('circuitId')['grid_end_diff'].transform('mean')
    
    # Create a temporary dataframe for the current year
    previous_years_data = previous_years_data[["overtakes_per_track", "circuitId"]].drop_duplicates()
    previous_years_data['year'] = year
    
    # Append the results for this year to the main dataframe
    df_overtakes = df_overtakes.append(previous_years_data, ignore_index=True)

# Merge the overtake information back into the original dataframe
data = data.merge(df_overtakes, on = ["circuitId", "year"], how  = "left")

C:\Users\jornv\AppData\Local\Temp/ipykernel_2440/1241654388.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  previous_years_data['overtakes_per_track'] = previous_years_data.groupby('circuitId')['grid_end_diff'].transform('mean')


In [18]:
# Sort the data by year, driverId and round
data = data.sort_values(['driverId', 'year', 'round'])

# Define a function to calculate the expanding mean
def expanding_mean(x):
    return x.expanding().mean()

# Group the data by driverId and year, then calculate the expanding mean for fastestLapSpeed and laptime_avg
data[['fastestLapSpeed_avg_driver', 'laptime_avg_avg_driver']] = (data.groupby(['driverId', 'year'])[['fastestLapSpeed', 'laptime_avg']].transform(expanding_mean))

# Group the data by year, then calculate the expanding mean for fastestLapSpeed and laptime_avg
data = data.sort_values(['year', 'round'])
data[['fastestLapSpeed_avg_year', 'laptime_avg_avg_year']] = data.groupby('year')[['fastestLapSpeed', 'laptime_avg']].transform(expanding_mean)

# Compute differences
data["fastestLapSpeedAvg_diff"] = data["fastestLapSpeed_avg_driver"] - data["fastestLapSpeed_avg_year"]
data["LaptimeAvg_diff"] = data["laptime_avg_avg_driver"] - data["laptime_avg_avg_year"]

In [19]:
# remove stop words
stop_words = {'Grand', 'Prix'}
f = lambda x: ' '.join(w for w in x.split() if not w in stop_words)
data["name_circuit"] = data["name_circuit"].apply(f)

# label homegame
data["homegame"] = (data['name_circuit'] == data["nationality_driver"]).astype(int)

# label homegame_constructor
data["homegame_constructor"] = (data['name_circuit'] == data["nationality_constructor"]).astype(int)

In [20]:
# Columns to be shifted
columns_to_shift = ["grid", "end_race_position", "overtakes_per_track", 
                    "diff_months", "homegame", "homegame_constructor"]

# Group the data
data_gr = data.groupby(['year', 'driverId'])[columns_to_shift]

# Shift the columns
data_shifted = data_gr.shift(periods=-1)

# Add "_t1" suffix to column names
data_shifted.columns = [f"{col}_t1" for col in data_shifted.columns]

# Join the shifted columns to the original dataframe
data = data.join(data_shifted)

In [21]:
data["diff_grid_standing"] = data["grid_t1"] - data["position_driverstanding"]

In [22]:
driverpoint = []
for index, row in data.iterrows():
    driverId = row['driverId']
    raceId = row['raceId']
    year = row['year']
    roundId = row["round"]
    driver_second = data[(data['driverId'] == driverId) & (data['year'] == year) & (data['round'] <= roundId) & (data['end_race_position'] == 2)]['end_race_position'].count()
    driver_third = data[(data['driverId'] == driverId) & (data['year'] == year) & (data['round'] <= roundId) & (data['end_race_position'] == 3)]['end_race_position'].count()
    driverpoint.append((raceId, driverId, driver_second, driver_third))
        
driverpoint_df = pd.DataFrame(driverpoint, columns =['raceId', 'driverId',  "driver_second", "driver_third"])
data =data.merge(driverpoint_df, on = ['raceId', 'driverId'])

In [23]:
conditions_top3 = [
    (data['end_race_position_t1'] <= 3),
    (data['end_race_position_t1'] > 3)
    ]

conditions_top2 = [
    (data['end_race_position_t1'] <= 2),
    (data['end_race_position_t1'] > 2)
    ]

conditions_top1 = [
    (data['end_race_position_t1'] <= 1),
    (data['end_race_position_t1'] > 1)
    ]

values = [1, 0]

data['end_race_position_t1_num'] = data['end_race_position_t1']
data['end_race_position_t1_top1'] = np.select(conditions_top1, values)
data['end_race_position_t1_top2'] = np.select(conditions_top2, values)
data['end_race_position_t1'] = np.select(conditions_top3, values)

In [24]:
# Calculate grid_end_diff
data_copy = data[data['raceId'] != 1114]

# Calculate grid_end_diff_abs
data["grid_end_diff_overtakes"] = (data_copy["end_race_position"] - data_copy["grid"]).astype(int).clip(lower=0).astype(int)

# Calculate grid_end_diff_abs
data["grid_end_diff_defense"] = (data_copy["end_race_position"] - data_copy["grid"]).astype(int).clip(upper=0).astype(int)

# Calculate the sum of driver standings per constructor, year, and race
total_driverstanding = data.groupby(['constructorId', 'raceId'])['position_driverstanding'].transform('sum')

# Subtract the current driver's standing from the total to get the teammate's standing
data['teammates_driverstanding'] = total_driverstanding - data['position_driverstanding']

# Sort the data by year, driverId and round
data = data.sort_values(['driverId', 'raceId'])

# Compute drivers_takeover_chance 
data[['drivers_takeover_chance']] = data.groupby('driverId')[['grid_end_diff_overtakes']].transform(expanding_mean)
data['drivers_takeover_chance'] = data['drivers_takeover_chance'].fillna(0)

# Calculate the sum of driver standings per constructor, year, and race
total_takeover = data.groupby(['constructorId', 'raceId'])['drivers_takeover_chance'].transform('sum')


# Subtract the current driver's standing from the total to get the teammate's standing
data['teammates_takeover_chance'] = total_takeover - data['drivers_takeover_chance']

# Compute drivers_defense_skills
data[['drivers_defense']] = data.groupby('driverId')[['grid_end_diff_defense']].transform(expanding_mean)
data['drivers_defense'] = data['drivers_defense'].fillna(0)
total_defense = data.groupby(['constructorId', 'raceId'])['drivers_defense'].transform('sum')
data['teammates_defense'] = total_defense - data['drivers_defense']

# Compute races_started
data['races_started'] = data.groupby(['driverId']).cumcount() + 1  # +1 to start counting from 1

In [25]:
for position in range(1, 6):  # Loop from 1 to 5
    # Filter the rows where the driver is in the current position in the standings
    in_standings = data[data['position_driverstanding'] == position]

    # Create a dictionary where each key is a raceId and the value is the grid position of the driver who is in the current position in the standings
    grid_positions = in_standings.set_index('raceId')['grid_t1'].to_dict()

    # Map this dictionary to a new column in the original dataframe
    data[f'grid_of_{position}_in_standings'] = data['raceId'].map(grid_positions)

In [26]:
data[data['year'] == 2023]

,resultId,raceId,driverId,constructorId,circuitId,grid,position,end_race_position,points_race,laps,...,drivers_takeover_chance,teammates_takeover_chance,drivers_defense,teammates_defense,races_started,grid_of_1_in_standings,grid_of_2_in_standings,grid_of_3_in_standings,grid_of_4_in_standings,grid_of_5_in_standings
26034,25850,1098,1,131,3,7,5.0,5,10.0,57,...,2.202572,1.855422,-1.308682,-1.650602,311,15.0,1.0,2.0,4.0,7.0
26054,25870,1099,1,131,77,7,5.0,5,10.0,50,...,2.195513,1.845238,-1.310897,-1.630952,312,1.0,0.0,4.0,5.0,3.0
26074,25887,1100,1,131,1,3,2.0,2,18.0,58,...,2.188498,2.011765,-1.309904,-1.611765,313,2.0,3.0,6.0,5.0,4.0
26094,25911,1101,1,131,73,5,6.0,6,8.0,51,...,2.184713,1.988372,-1.305732,-1.627907,314,9.0,1.0,2.0,13.0,3.0
26114,25931,1102,1,131,79,13,6.0,6,8.0,57,...,2.177778,1.965517,-1.323810,-1.632184,315,1.0,20.0,2.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26293,26098,1111,859,213,39,19,13.0,13,0.0,72,...,0.000000,2.947368,-6.000000,-1.701754,1,2.0,5.0,10.0,8.0,1.0
26313,26116,1112,859,213,14,12,11.0,11,0.0,51,...,0.000000,3.051724,-3.500000,-1.672414,2,11.0,13.0,7.0,5.0,1.0
26333,26134,1113,859,213,15,10,9.0,9,2.0,62,...,0.000000,3.067797,-2.666667,-1.644068,3,1.0,5.0,7.0,10.0,6.0
26353,26156,1114,859,213,22,11,11.0,11,0.0,52,...,0.000000,3.067797,-2.666667,-1.644068,4,1.0,0.0,3.0,4.0,12.0


In [29]:
data.to_csv(r'C:\Users\jornv\OneDrive\Documenten\Projects\F1\Data\Input\Formula1_ML.csv')

In [28]:
data[data['year'] == 2023].sort_values(['resultId'])

,resultId,raceId,driverId,constructorId,circuitId,grid,position,end_race_position,points_race,laps,...,drivers_takeover_chance,teammates_takeover_chance,drivers_defense,teammates_defense,races_started,grid_of_1_in_standings,grid_of_2_in_standings,grid_of_3_in_standings,grid_of_4_in_standings,grid_of_5_in_standings
26040,25846,1098,830,9,3,1,1.0,1,25.0,57,...,2.743902,2.163866,-1.621951,-2.256303,164,15.0,1.0,2.0,4.0,7.0
26037,25847,1098,815,9,3,2,2.0,2,18.0,57,...,2.163866,2.743902,-2.256303,-1.621951,238,15.0,1.0,2.0,4.0,7.0
26035,25848,1098,4,117,3,5,3.0,3,15.0,57,...,1.972145,1.284553,-2.161560,-2.707317,359,15.0,1.0,2.0,4.0,7.0
26041,25849,1098,832,6,3,4,4.0,4,12.0,57,...,2.469512,2.528846,-1.945122,-1.288462,164,15.0,1.0,2.0,4.0,7.0
26034,25850,1098,1,131,3,7,5.0,5,10.0,57,...,2.202572,1.855422,-1.308682,-1.650602,311,15.0,1.0,2.0,4.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26473,26281,1120,858,3,24,20,16.0,16,0.0,58,...,1.380952,1.787500,-2.285714,-1.925000,22,NaN,NaN,NaN,NaN,NaN
26471,26282,1120,855,51,24,19,17.0,17,0.0,58,...,2.139535,2.355856,-1.883721,-1.315315,44,NaN,NaN,NaN,NaN,NaN
26462,26283,1120,832,6,24,16,18.0,18,0.0,57,...,2.505435,2.741935,-1.831522,-1.233871,185,NaN,NaN,NaN,NaN,NaN
26459,26284,1120,822,51,24,18,19.0,19,0.0,57,...,2.355856,2.139535,-1.315315,-1.883721,223,NaN,NaN,NaN,NaN,NaN
